## Importing relevant libraries

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

## Construct the Schlogl operator

In [3]:
def TridiagA(lambda_func, mu_func, N):
    """
    Creates tridiagonal stochastic matrix for the Schlogl model
    Inputs lambda and mu model functions, and N desired size of matrix
    Returns stochastic matrix (non-hermitian)
    """
    # initialize diagonals
    d1 = np.zeros(N - 1)
    d2 = np.zeros(N)
    d3 = np.zeros(N - 1)

    # element 1,1
    d2[0] = -lambda_func(0)

    # element N,N
    d2[N - 1] = -mu_func(N - 1)

    # bottom diagonal elements
    for i in range(N - 1):
        d1[i] = lambda_func(i)

    # top diagonal elements
    for i in range(1, N):
        d3[i - 1] = mu_func(i)

    # main diagonal elements
    for i in range(1, N - 1):
        d2[i] = -lambda_func(i) - mu_func(i)

    # putting the diagonals together
    Q = np.diag(d1, k = -1) + np.diag(d2, k = 0) + np.diag(d3, k = 1)
    
    return Q

def get_volume_array(start_V, stop_V, n_operator_qubits):
    # Function to generate the initial volume array (to carry out computations)
    
    num_elements = 2 ** n_operator_qubits
    step_size = (stop_V - start_V)/(num_elements - 1)
    
    # Generate the volume array, given that the step size has been determined
    volume_array = np.arange(start_V, stop_V, step_size)
    
    return volume_array

def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)
    
    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes

    # Get absolute values
    b = np.abs(b)
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_rmsd(list1, list2):
    # Ensure the lists have the same length
    
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same length.")
    
    # Compute the RMSD
    rmsd = np.sqrt(np.mean((np.array(list1) - np.array(list2)) ** 2))
    return rmsd

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

## VQD implementation

In [6]:
# Define the VQD computation with varying ansatz depth
def run_vqd(matrix, ansatz, optimizer, seed, zeromode_exact):
    # Function to compute the execution time for different optimizers and ansatz depths (averaged over 10 independent VQD runs in each case)

    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Define the qubit Hamiltonian
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)

    # Compute using NumPyMinimumEigensolver for reference
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qub_hamiltonian)
    
    # Initial point for the classical optimizer
    seed_value = seed
    np.random.seed(seed_value)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    
    # Initializing the Estimator, Sampler, and fidelity
    estimator = Estimator()
    sampler = Sampler()
    fidelity = ComputeUncompute(sampler)
    
    # Run the VQE algorithm
    @dataclass
    class VQDLog:
        values: list = None
        parameters: list = None
        
        def __post_init__(self):
            self.values = []
            self.parameters = []

        # Update function to match the expected arguments
        def update(self, count, parameters, mean, _metadata, _extra):
            self.values.append(mean)
            self.parameters.append(parameters)

    log = VQDLog()

    vqd = VQD(estimator,
              fidelity, ansatz, optimizer, k=1, initial_point=initial_point, callback=log.update)
    result = vqd.compute_eigenvalues(qub_hamiltonian)

    # Get the optimizer runtime
    time = result.optimizer_times

    # Estimate the zeromode
    optimal_params = result.optimal_points
    zeromode_points = optimal_params[0]
    final_circuit = ansatz.assign_parameters(zeromode_points)
    zeromode_vqd = Statevector.from_instruction(final_circuit)
    zeromode = zeromode_vqd.data.tolist()

    # Compute the fidelity between the classical and quantum zeromodes
    zeromode_exact = np.array(zeromode_exact).reshape(len(zeromode_exact), 1)
    zeromode = np.array(zeromode).reshape(len(zeromode_exact), 1)
    fidelity = get_similarity(zeromode_exact, zeromode)

    # Get the number of function calls
    num_func_calls = len(log.values)

    return zeromode, fidelity, num_func_calls

### Get the matrix and the zeromode

In [9]:
def get_zeromode(num_operator_qubits, V):
    # Function to get the matrix and the zeromode

    ## Computing the block diagonal representation of the Schlogl operator matrix
    # Defining parameters
    a = 1
    b = 1
    k1 = 3
    k2 = 0.6
    k3 = 0.25
    k4 = 2.95
    
    # Number of qubits
    num_operator_qubits = num_operator_qubits
    
    # Matrix dimensions
    N = 2 ** num_operator_qubits
    
    # Generating the basis size array
    x_axis = []
    
    for i in range(N):
        x_axis.append(i)
    
    # # Constructing the Schlogl operator for V = 1.1
    # # Get the volume array
    # start_V = 0.1
    # stop_V = 1.6
    # volume_array = get_volume_array(start_V, stop_V, num_operator_qubits)
    
    # # For system volume V = 1.1
    # volume_array = np.arange(0.1, 10.6, 0.1)
    
    # # Construct the matrix representation of the operator
    # for i, V in enumerate(volume_array):
        
    #     # Birth and death rates
    #     lambda_fn = lambda n: ((a*k1*n*(n-1))/V + b*k3*V)
    #     mu_fn = lambda n: ((k2*n*(n-1)*(n-2))/V**2 + n*k4)
    
    #     # stochastic matrix Q of dimension N x N
    #     Q = TridiagA(lambda_fn, mu_fn, N)
        
    #     i += 1
    # ######################################################################  
    # Construct the matrix
    # Birth and death rates
    lambda_fn = lambda n: ((a*k1*n*(n-1))/V + b*k3*V)
    mu_fn = lambda n: ((k2*n*(n-1)*(n-2))/V**2 + n*k4)

    # stochastic matrix Q of dimension N x N
    Q = TridiagA(lambda_fn, mu_fn, N)
    
    # Print the original Schlogl operator matrix
    print('The Schlogl operator matrix is:')
    print(Q)
    print()
        
    # Compute the Hermitian form of the matrix
    hermitian_matrix = np.dot(Q.T, Q)
    
    # # Print the volume array
    # print('The volume array is:')
    # print(volume_array)
    # print()
    
    # Print the volume parameter for which the simulations are being run
    print('The volume parameter is:')
    print(V)
    print()
    
    # Print the Hermitian matrix (block diagonal form)
    print('The Hermitian form of the Schlogl matrix is:')
    print(hermitian_matrix)
    print()
    
   ## Get the classical zeromode
    A, P = la.eig(Q)
    
    A_real = np.real(A)
    index = np.where(A_real == np.amax(A_real))[0][0]
    
    eigenvalue = A[index]
    zeromode_classic = P[:, index]
    
    zeromode_classic = np.real(normalize_eigenvector(zeromode_classic))
    zeromode_classic = np.reshape(zeromode_classic, (zeromode_classic.size, 1))
    
    print("the available eigenvalues are: \n" + str(A))
    print()
    
    print("The minimum eigenvalue located is: \n" + str(eigenvalue))
    print()
    
    print("The minimum zeromode located is: \n" + str(np.real(zeromode_classic)))

    return Q, hermitian_matrix, zeromode_classic

## Get resource estimates

In [12]:
def estimate_resources(matrix, zeromode_classic, optimizers, num_qubits, target_fidelity_threshold):
    
    # Initial depth and maximum depth
    initial_depth = 1
    max_depth = 10  # Set the maximum number of depths to check

    # Initialize dictionaries to store minimum depths, zeromodes, and resources for each optimizer-ansatz pair
    min_depths = {}
    zeromodes = {}
    resource_info = {}  # To store gate count, circuit depth, and function calls
    
    # Loop through each optimizer
    for optimizer in optimizers:
        optimizer_name = optimizer.__class__.__name__
        
        # Loop through each ansatz type and configure dynamically
        for AnsatzClass in [RealAmplitudes, TwoLocal, EfficientSU2]:
            # Initialize ansatz with appropriate configurations
            if AnsatzClass == RealAmplitudes:
                ansatz = AnsatzClass(num_qubits=num_qubits, entanglement='full', reps=initial_depth)
            elif AnsatzClass == TwoLocal:
                ansatz = AnsatzClass(num_qubits=num_qubits, rotation_blocks=['ry'], entanglement_blocks='cx', reps=initial_depth)
            elif AnsatzClass == EfficientSU2:
                ansatz = AnsatzClass(num_qubits=num_qubits, su2_gates=['ry'], entanglement='sca', reps=initial_depth)
            
            ansatz_name = AnsatzClass.__name__
            pair_name = f"{optimizer_name}-{ansatz_name}"
            print(f"\nRunning VQD for optimizer-ansatz pair: {pair_name}")
            
            current_depth = initial_depth
            converged = False  # Flag to check if convergence occurs
            
            while current_depth <= max_depth:  # Loop for up to max_depth
                # Set `reps` (depth) for the current ansatz
                ansatz.reps = current_depth
                
                # Temporary storage for the fidelity results
                fidelities = []
                function_calls = []  # To track the number of function calls for each run
                all_zeromodes = []  # To store all zeromodes for the depth
                
                # Perform multiple independent VQE runs to calculate average fidelity
                for run in range(10):  # Number of independent runs
                    # Set a seed for a specific VQE run
                    seed = run + 1
                    
                    # Run VQE
                    zeromode, fidelity_value, function_call_count = run_vqd(
                        matrix=matrix, ansatz=ansatz, optimizer=optimizer, seed=seed,
                        zeromode_exact=zeromode_classic)
                    
                    # Append the fidelity result and function call count
                    fidelities.append(fidelity_value)
                    function_calls.append(function_call_count)
                    all_zeromodes.append(zeromode)
                
                # Calculate the average fidelity over the runs
                average_fidelity = np.mean(fidelities)
                print(f"{pair_name} - Depth {current_depth}: Average fidelity = {average_fidelity}")
        
                # Check if the average fidelity meets the threshold
                if average_fidelity >= target_fidelity_threshold:
                    min_depths[pair_name] = current_depth
                    converged = True
                    
                    # Identify the run with the highest fidelity at this depth
                    best_run_index = np.argmax(fidelities)
                    best_fidelity = fidelities[best_run_index]
                    best_zeromode = all_zeromodes[best_run_index]
                    best_function_calls = function_calls[best_run_index]
                    
                    # Print the run number and highest fidelity at optimal depth
                    print(f"Optimal depth {current_depth} for {pair_name} achieved highest fidelity = {best_fidelity}")
                    print(f"Run number with highest fidelity: {best_run_index + 1}")
                    
                    # Calculate gate count and circuit depth for the ansatz at this depth
                    decomposed_ansatz = ansatz.decompose()  # Decompose to get actual gate operations
                    gate_count_dict = decomposed_ansatz.count_ops()
                    total_gates = sum(gate_count_dict.values())
                    circuit_depth = decomposed_ansatz.depth()
                    
                    # Store zeromode and resource information
                    zeromodes[pair_name] = best_zeromode
                    resource_info[pair_name] = {
                        'gate_count': total_gates,
                        'circuit_depth': circuit_depth,
                        'function_calls': best_function_calls
                    }
                    
                    print(f"Zeromode at optimal fidelity for {pair_name}: {best_zeromode}")
                    print(f"Resource estimates for {pair_name}: Gate count = {total_gates}, Circuit depth = {circuit_depth}, Function calls = {best_function_calls}")
                    
                    break  # Exit the loop if the threshold is met
        
                current_depth += 1  # Increase depth and try again
            
            # If the loop finishes and no convergence occurs, mark as "did not converge"
            if not converged:
                min_depths[pair_name] = "Did not converge"
                zeromodes[pair_name] = "Did not converge"
                resource_info[pair_name] = {
                    'gate_count': "N/A",
                    'circuit_depth': "N/A",
                    'function_calls': "N/A"
                }
                print(f"{pair_name} did not converge within {max_depth} depths.")
    
    return min_depths, zeromodes, resource_info

## Quantum (VQD) analysis

### For V = 1.1 (N = 2)

### Get the matrix and the zeromode

In [22]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(2, 1.1)

The Schlogl operator matrix is:
[[ -0.275        2.95         0.           0.        ]
 [  0.275       -3.225        5.9          0.        ]
 [  0.           0.275      -11.62954545  11.82520661]
 [  0.           0.           5.72954545 -11.82520661]]

The volume parameter is:
1.1

The Hermitian form of the Schlogl matrix is:
[[ 1.51250000e-01 -1.69812500e+00  1.62250000e+00  0.00000000e+00]
 [-1.69812500e+00  1.91787500e+01 -2.22256250e+01  3.25193182e+00]
 [ 1.62250000e+00 -2.22256250e+01  2.02884019e+02 -2.05274837e+02]
 [ 0.00000000e+00  3.25193182e+00 -2.05274837e+02  2.79671023e+02]]

the available eigenvalues are: 
[  0.        +0.j  -2.60510311+0.j  -4.34244163+0.j -20.00720732+0.j]

The minimum eigenvalue located is: 
0j

The minimum zeromode located is: 
[[0.99567159]
 [0.09281684]
 [0.00432621]
 [0.00209613]]


### Estimate resources (fix at 5000 optimization iterations)

In [31]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter), NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
target_fidelity_threshold = 0.99

# Estimate min. ansatz depths
min_depths_N_2, zeromodes_N_2, resources_N_2 = estimate_resources(hermitian_matrix, zeromode, optimizers, num_qubits, target_fidelity_threshold)


Running VQD for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


SLSQP-RealAmplitudes - Depth 1: Average fidelity = 0.8990441096655062
SLSQP-RealAmplitudes - Depth 2: Average fidelity = 0.9999999972097704
Optimal depth 2 for SLSQP-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 7
Zeromode at optimal fidelity for SLSQP-RealAmplitudes: [[0.99567112+0.j]
 [0.09282169+0.j]
 [0.00432976+0.j]
 [0.00209785+0.j]]
Resource estimates for SLSQP-RealAmplitudes: Gate count = 8, Circuit depth = 5, Function calls = 90

Running VQD for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average fidelity = 0.999999993238301
Optimal depth 1 for SLSQP-TwoLocal achieved highest fidelity = [[1.]]
Run number with highest fidelity: 8
Zeromode at optimal fidelity for SLSQP-TwoLocal: [[0.99567147+0.j]
 [0.09281813+0.j]
 [0.00432705+0.j]
 [0.00209685+0.j]]
Resource estimates for SLSQP-TwoLocal: Gate count = 5, Circuit depth = 3, Function calls = 153

Running VQD for optimizer-ansatz pair: SLSQP-EfficientSU2
SLSQP-EfficientSU2 -

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 2: Average fidelity = 0.9991331379498973
Optimal depth 2 for SLSQP-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 8
Zeromode at optimal fidelity for SLSQP-EfficientSU2: [[-0.99567146+0.j]
 [-0.09281834+0.j]
 [-0.00432563+0.j]
 [-0.00209491+0.j]]
Resource estimates for SLSQP-EfficientSU2: Gate count = 8, Circuit depth = 5, Function calls = 61

Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-RealAmplitudes - Depth 1: Average fidelity = 0.9981787039620349
Optimal depth 1 for P_BFGS-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 7
Zeromode at optimal fidelity for P_BFGS-RealAmplitudes: [[-0.9956716 +0.j]
 [-0.09281682+0.j]
 [-0.00432616+0.j]
 [-0.00209605+0.j]]
Resource estimates for P_BFGS-RealAmplitudes: Gate count = 5, Circuit depth = 3, Function calls = 135

Running VQD for optimizer-ansatz pair: P_BFGS-TwoLocal


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-TwoLocal - Depth 1: Average fidelity = 0.9999999999999372
Optimal depth 1 for P_BFGS-TwoLocal achieved highest fidelity = [[1.]]
Run number with highest fidelity: 5
Zeromode at optimal fidelity for P_BFGS-TwoLocal: [[-0.9956716 +0.j]
 [-0.09281683+0.j]
 [-0.00432621+0.j]
 [-0.00209614+0.j]]
Resource estimates for P_BFGS-TwoLocal: Gate count = 5, Circuit depth = 3, Function calls = 140

Running VQD for optimizer-ansatz pair: P_BFGS-EfficientSU2


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 1: Average fidelity = 0.9981787039620349
Optimal depth 1 for P_BFGS-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 7
Zeromode at optimal fidelity for P_BFGS-EfficientSU2: [[-0.9956716 +0.j]
 [-0.09281682+0.j]
 [-0.00432616+0.j]
 [-0.00209605+0.j]]
Resource estimates for P_BFGS-EfficientSU2: Gate count = 5, Circuit depth = 3, Function calls = 135

Running VQD for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes
NELDER_MEAD-RealAmplitudes - Depth 1: Average fidelity = 0.9999999997343855
Optimal depth 1 for NELDER_MEAD-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 2
Zeromode at optimal fidelity for NELDER_MEAD-RealAmplitudes: [[0.9956721 +0.j]
 [0.09281153+0.j]
 [0.00432379+0.j]
 [0.0020948 +0.j]]
Resource estimates for NELDER_MEAD-RealAmplitudes: Gate count = 5, Circuit depth = 3, Function calls = 423

Running VQD for optimizer-ansatz pair: NELDER_MEAD-TwoLocal
NELDER_MEAD-TwoLocal - Dep

### Print resource estimates

In [34]:
## Print the min. ansatz depths
print('The min. ansatz depths are:')
print(min_depths_N_2)
print()

## Print the resources required
print('The resources required are:')
print(resources_N_2)
print()

The min. ansatz depths are:
{'SLSQP-RealAmplitudes': 2, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 2, 'P_BFGS-RealAmplitudes': 1, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 1, 'NELDER_MEAD-RealAmplitudes': 1, 'NELDER_MEAD-TwoLocal': 1, 'NELDER_MEAD-EfficientSU2': 1, 'POWELL-RealAmplitudes': 4, 'POWELL-TwoLocal': 1, 'POWELL-EfficientSU2': 4}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 8, 'circuit_depth': 5, 'function_calls': 90}, 'SLSQP-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 153}, 'SLSQP-EfficientSU2': {'gate_count': 8, 'circuit_depth': 5, 'function_calls': 61}, 'P_BFGS-RealAmplitudes': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 135}, 'P_BFGS-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 140}, 'P_BFGS-EfficientSU2': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 135}, 'NELDER_MEAD-RealAmplitudes': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 423}, 'NELDER_MEAD-TwoLocal': {'gate_c

### Compute expectation values in the ground state

In [37]:
# Get the zeromodes
zeromodes = list(zeromodes_N_2.values())

# Initialize the list to store real-valued results
zeromodes_true = []

# Iterate through each entry in zeromodes
for arr in zeromodes:
    if isinstance(arr, str) and arr == "Did not converge":
        # Handle the case where the entry is the string 'Did not converge'
        zeromodes_true.append('Did not converge')
    else:
        # Convert the array to real numbers if it's not a string
        zeromodes_true.append([elem.real for elem in arr.flatten()])

# Initialize
expectation_values = []

# Compute 
for zeromode in zeromodes_true:
    if zeromode != "Did not converge":
        
        # Compute the expectation value in the ground state
        value = get_expectation(Q, np.abs(zeromode))
        expectation_values.append(value)
    else:
        expectation_values.append('No solution')

# Get the expectation values for the individual optimizers
expectation_slsqp = expectation_values[:3]
expectation_bfgs = expectation_values[3:6]
expectation_nelder = expectation_values[6:9]
expectation_powell = expectation_values[9:12]

## Print the expectation values
print('The expectation values for SLSQP are:')
print(expectation_slsqp)
print()

print('The expectation values for P-BFGS are:')
print(expectation_bfgs)
print()

print('The expectation values for NELDER-MEAD are:')
print(expectation_nelder)
print()

print('The expectation values for POWELL are:')
print(expectation_powell)
print()

The expectation values for SLSQP are:
[array([[1.47590866e-05]]), array([[3.86938069e-06]]), array([[3.65833805e-06]])]

The expectation values for P-BFGS are:
[array([[-1.01942786e-07]]), array([[-2.23637969e-08]]), array([[-1.01942786e-07]])]

The expectation values for NELDER-MEAD are:
[array([[-1.54290939e-05]]), array([[-1.54290939e-05]]), array([[-1.54290939e-05]])]

The expectation values for POWELL are:
[array([[6.57286142e-07]]), array([[-3.59813277e-08]]), array([[1.2512823e-06]])]



### For V = 5.5 (N = 2)

### Get the matrix and the zeromode

In [40]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(2, 5.5)

The Schlogl operator matrix is:
[[-1.375       2.95        0.          0.        ]
 [ 1.375      -4.325       5.9         0.        ]
 [ 0.          1.375      -8.36590909  8.96900826]
 [ 0.          0.          2.46590909 -8.96900826]]

The volume parameter is:
5.5

The Hermitian form of the Schlogl matrix is:
[[  3.78125    -10.003125     8.1125       0.        ]
 [-10.003125    29.29875    -37.020625    12.33238636]
 [  8.1125     -37.020625   110.87914256 -97.15066679]
 [  0.          12.33238636 -97.15066679 160.8862185 ]]

the available eigenvalues are: 
[ 4.44089210e-16+0.j -2.84354832e+00+0.j -6.37631943e+00+0.j
 -1.38150496e+01+0.j]

The minimum eigenvalue located is: 
(4.440892098500626e-16+0j)

The minimum zeromode located is: 
[[-0.90169082]
 [-0.42027962]
 [-0.09794652]
 [-0.02692909]]


### Estimate resources (fix at 5000 optimization iterations)

In [44]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter), NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
target_fidelity_threshold = 0.99

# Estimate min. ansatz depths
min_depths_N_2, zeromodes_N_2, resources_N_2 = estimate_resources(hermitian_matrix, zeromode, optimizers, num_qubits, target_fidelity_threshold)


Running VQD for optimizer-ansatz pair: SLSQP-RealAmplitudes
SLSQP-RealAmplitudes - Depth 1: Average fidelity = 0.9813940914956614
SLSQP-RealAmplitudes - Depth 2: Average fidelity = 0.9325535661168823
SLSQP-RealAmplitudes - Depth 3: Average fidelity = 0.9985387034982814
Optimal depth 3 for SLSQP-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 10
Zeromode at optimal fidelity for SLSQP-RealAmplitudes: [[0.90169182+0.j]
 [0.42027809+0.j]
 [0.09794429+0.j]
 [0.02692779+0.j]]
Resource estimates for SLSQP-RealAmplitudes: Gate count = 11, Circuit depth = 7, Function calls = 131

Running VQD for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average fidelity = 0.9999999945835869
Optimal depth 1 for SLSQP-TwoLocal achieved highest fidelity = [[1.]]
Run number with highest fidelity: 5
Zeromode at optimal fidelity for SLSQP-TwoLocal: [[0.90169431+0.j]
 [0.42027197+0.j]
 [0.09794674+0.j]
 [0.02693104+0.j]]
Resource estimates for SLSQP-TwoLocal: 

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 3: Average fidelity = 0.9999999935092024
Optimal depth 3 for SLSQP-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 8
Zeromode at optimal fidelity for SLSQP-EfficientSU2: [[0.90168947+0.j]
 [0.42028241+0.j]
 [0.09794587+0.j]
 [0.0269335 +0.j]]
Resource estimates for SLSQP-EfficientSU2: Gate count = 11, Circuit depth = 7, Function calls = 130

Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Average fidelity = 0.9828015656145054


...

P_BFGS-EfficientSU2 - Depth 3: Average fidelity = 0.982898840546498


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 4: Average fidelity = 0.948226835661746


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Average fidelity = 0.9999999999999719
Optimal depth 5 for P_BFGS-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 2
Zeromode at optimal fidelity for P_BFGS-EfficientSU2: [[0.90169081+0.j]
 [0.42027965+0.j]
 [0.09794653+0.j]
 [0.02692909+0.j]]
Resource estimates for P_BFGS-EfficientSU2: Gate count = 17, Circuit depth = 11, Function calls = 169

Running VQD for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes
NELDER_MEAD-RealAmplitudes - Depth 1: Average fidelity = 0.9999999996794205
Optimal depth 1 for NELDER_MEAD-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 5
Zeromode at optimal fidelity for NELDER_MEAD-RealAmplitudes: [[0.90169159+0.j]
 [0.42027942+0.j]
 [0.09794105+0.j]
 [0.02692645+0.j]]
Resource estimates for NELDER_MEAD-RealAmplitudes: Gate count = 5, Circuit depth = 3, Function calls = 181

Running VQD for optimizer-ansatz pair: NELDER_MEAD-TwoLocal
NELDER_MEAD-TwoLocal - Depth

### Print resource estimates

In [47]:
## Print the min. ansatz depths
print('The min. ansatz depths are:')
print(min_depths_N_2)
print()

## Print the resources required
print('The resources required are:')
print(resources_N_2)
print()

The min. ansatz depths are:
{'SLSQP-RealAmplitudes': 3, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 3, 'P_BFGS-RealAmplitudes': 3, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 5, 'NELDER_MEAD-RealAmplitudes': 1, 'NELDER_MEAD-TwoLocal': 1, 'NELDER_MEAD-EfficientSU2': 1, 'POWELL-RealAmplitudes': 5, 'POWELL-TwoLocal': 1, 'POWELL-EfficientSU2': 2}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 11, 'circuit_depth': 7, 'function_calls': 131}, 'SLSQP-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 75}, 'SLSQP-EfficientSU2': {'gate_count': 11, 'circuit_depth': 7, 'function_calls': 130}, 'P_BFGS-RealAmplitudes': {'gate_count': 11, 'circuit_depth': 7, 'function_calls': 144}, 'P_BFGS-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 95}, 'P_BFGS-EfficientSU2': {'gate_count': 17, 'circuit_depth': 11, 'function_calls': 169}, 'NELDER_MEAD-RealAmplitudes': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 181}, 'NELDER_MEAD-TwoLocal': {'g

### Compute expectation values in the ground state

In [50]:
# Get the zeromodes
zeromodes = list(zeromodes_N_2.values())

# Initialize the list to store real-valued results
zeromodes_true = []

# Iterate through each entry in zeromodes
for arr in zeromodes:
    if isinstance(arr, str) and arr == "Did not converge":
        # Handle the case where the entry is the string 'Did not converge'
        zeromodes_true.append('Did not converge')
    else:
        # Convert the array to real numbers if it's not a string
        zeromodes_true.append([elem.real for elem in arr.flatten()])

# Initialize
expectation_values = []

# Compute 
for zeromode in zeromodes_true:
    if zeromode != "Did not converge":
        
        # Compute the expectation value in the ground state
        value = get_expectation(Q, np.abs(zeromode))
        expectation_values.append(value)
    else:
        expectation_values.append('No solution')

# Get the expectation values for the individual optimizers
expectation_slsqp = expectation_values[:3]
expectation_bfgs = expectation_values[3:6]
expectation_nelder = expectation_values[6:9]
expectation_powell = expectation_values[9:12]

## Print the expectation values
print('The expectation values for SLSQP are:')
print(expectation_slsqp)
print()

print('The expectation values for P-BFGS are:')
print(expectation_bfgs)
print()

print('The expectation values for NELDER-MEAD are:')
print(expectation_nelder)
print()

print('The expectation values for POWELL are:')
print(expectation_powell)
print()

The expectation values for SLSQP are:
[array([[-6.84406188e-06]]), array([[-8.15942788e-06]]), array([[5.30984756e-06]])]

The expectation values for P-BFGS are:
[array([[-6.35677962e-08]]), array([[4.37160805e-09]]), array([[4.80653133e-08]])]

The expectation values for NELDER-MEAD are:
[array([[-1.18406504e-05]]), array([[-1.18406504e-05]]), array([[-1.18406504e-05]])]

The expectation values for POWELL are:
[array([[1.56631202e-06]]), array([[1.23320874e-08]]), array([[1.67647263e-06]])]



### For V = 10.5 (N = 2)

### Get the matrix and the zeromode

In [53]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(2, 10.5)

The Schlogl operator matrix is:
[[-2.625       2.95        0.          0.        ]
 [ 2.625      -5.575       5.9         0.        ]
 [ 0.          2.625      -9.09642857  8.88265306]
 [ 0.          0.          3.19642857 -8.88265306]]

The volume parameter is:
10.5

The Hermitian form of the Schlogl matrix is:
[[  13.78125     -22.378125     15.4875        0.        ]
 [ -22.378125     46.67375     -56.770625     23.31696429]
 [  15.4875      -56.770625    127.77216837 -109.19318513]
 [   0.           23.31696429 -109.19318513  157.80305081]]

the available eigenvalues are: 
[-1.52577371e+01+0.j -7.74424342e+00+0.j -9.70174270e-16+0.j
 -3.17710106e+00+0.j]

The minimum eigenvalue located is: 
(-9.701742699678762e-16+0j)

The minimum zeromode located is: 
[[-0.71268205]
 [-0.63416623]
 [-0.28215023]
 [-0.10153195]]


### Estimate resources (fix at 5000 optimization iterations)

In [56]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter), NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
target_fidelity_threshold = 0.99

# Estimate min. ansatz depths
min_depths_N_2, zeromodes_N_2, resources_N_2 = estimate_resources(hermitian_matrix, zeromode, optimizers, num_qubits, target_fidelity_threshold)


Running VQD for optimizer-ansatz pair: SLSQP-RealAmplitudes
SLSQP-RealAmplitudes - Depth 1: Average fidelity = 0.9999999961275646
Optimal depth 1 for SLSQP-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 10
Zeromode at optimal fidelity for SLSQP-RealAmplitudes: [[0.71269066+0.j]
 [0.63415874+0.j]
 [0.28214425+0.j]
 [0.10153489+0.j]]
Resource estimates for SLSQP-RealAmplitudes: Gate count = 5, Circuit depth = 3, Function calls = 98

Running VQD for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average fidelity = 0.9999999974534421
Optimal depth 1 for SLSQP-TwoLocal achieved highest fidelity = [[1.]]
Run number with highest fidelity: 8
Zeromode at optimal fidelity for SLSQP-TwoLocal: [[0.71268429+0.j]
 [0.63416361+0.j]
 [0.28215041+0.j]
 [0.10153207+0.j]]
Resource estimates for SLSQP-TwoLocal: Gate count = 5, Circuit depth = 3, Function calls = 90

Running VQD for optimizer-ansatz pair: SLSQP-EfficientSU2


For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 1: Average fidelity = 0.9999999961275646
Optimal depth 1 for SLSQP-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 10
Zeromode at optimal fidelity for SLSQP-EfficientSU2: [[0.71269066+0.j]
 [0.63415874+0.j]
 [0.28214425+0.j]
 [0.10153489+0.j]]
Resource estimates for SLSQP-EfficientSU2: Gate count = 5, Circuit depth = 3, Function calls = 98

Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-RealAmplitudes - Depth 1: Average fidelity = 0.9999999999999446
Optimal depth 1 for P_BFGS-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 5
Zeromode at optimal fidelity for P_BFGS-RealAmplitudes: [[0.71268207+0.j]
 [0.63416621+0.j]
 [0.28215022+0.j]
 [0.10153194+0.j]]
Resource estimates for P_BFGS-RealAmplitudes: Gate count = 5, Circuit depth = 3, Function calls = 150

Running VQD for optimizer-ansatz pair: P_BFGS-TwoLocal


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-TwoLocal - Depth 1: Average fidelity = 0.9999999999999742
Optimal depth 1 for P_BFGS-TwoLocal achieved highest fidelity = [[1.]]
Run number with highest fidelity: 2
Zeromode at optimal fidelity for P_BFGS-TwoLocal: [[-0.71268208+0.j]
 [-0.6341662 +0.j]
 [-0.28215022+0.j]
 [-0.10153194+0.j]]
Resource estimates for P_BFGS-TwoLocal: Gate count = 5, Circuit depth = 3, Function calls = 105

Running VQD for optimizer-ansatz pair: P_BFGS-EfficientSU2


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 1: Average fidelity = 0.9999999999999446
Optimal depth 1 for P_BFGS-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 5
Zeromode at optimal fidelity for P_BFGS-EfficientSU2: [[0.71268207+0.j]
 [0.63416621+0.j]
 [0.28215022+0.j]
 [0.10153194+0.j]]
Resource estimates for P_BFGS-EfficientSU2: Gate count = 5, Circuit depth = 3, Function calls = 150

Running VQD for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes
NELDER_MEAD-RealAmplitudes - Depth 1: Average fidelity = 0.9999999994139251
Optimal depth 1 for NELDER_MEAD-RealAmplitudes achieved highest fidelity = [[1.]]
Run number with highest fidelity: 6
Zeromode at optimal fidelity for NELDER_MEAD-RealAmplitudes: [[-0.71268421+0.j]
 [-0.63416434+0.j]
 [-0.28214926+0.j]
 [-0.10153134+0.j]]
Resource estimates for NELDER_MEAD-RealAmplitudes: Gate count = 5, Circuit depth = 3, Function calls = 218

Running VQD for optimizer-ansatz pair: NELDER_MEAD-TwoLocal
NELDER_MEAD-TwoLocal - Dep

### Print resource estimates

In [59]:
## Print the min. ansatz depths
print('The min. ansatz depths are:')
print(min_depths_N_2)
print()

## Print the resources required
print('The resources required are:')
print(resources_N_2)
print()

The min. ansatz depths are:
{'SLSQP-RealAmplitudes': 1, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 1, 'P_BFGS-RealAmplitudes': 1, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 1, 'NELDER_MEAD-RealAmplitudes': 1, 'NELDER_MEAD-TwoLocal': 1, 'NELDER_MEAD-EfficientSU2': 1, 'POWELL-RealAmplitudes': 2, 'POWELL-TwoLocal': 1, 'POWELL-EfficientSU2': 2}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 98}, 'SLSQP-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 90}, 'SLSQP-EfficientSU2': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 98}, 'P_BFGS-RealAmplitudes': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 150}, 'P_BFGS-TwoLocal': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 105}, 'P_BFGS-EfficientSU2': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 150}, 'NELDER_MEAD-RealAmplitudes': {'gate_count': 5, 'circuit_depth': 3, 'function_calls': 218}, 'NELDER_MEAD-TwoLocal': {'gate_co

### Compute expectation values in the ground state

In [62]:
# Get the zeromodes
zeromodes = list(zeromodes_N_2.values())

# Initialize the list to store real-valued results
zeromodes_true = []

# Iterate through each entry in zeromodes
for arr in zeromodes:
    if isinstance(arr, str) and arr == "Did not converge":
        # Handle the case where the entry is the string 'Did not converge'
        zeromodes_true.append('Did not converge')
    else:
        # Convert the array to real numbers if it's not a string
        zeromodes_true.append([elem.real for elem in arr.flatten()])

# Initialize
expectation_values = []

# Compute 
for zeromode in zeromodes_true:
    if zeromode != "Did not converge":
        
        # Compute the expectation value in the ground state
        value = get_expectation(Q, np.abs(zeromode))
        expectation_values.append(value)
    else:
        expectation_values.append('No solution')

# Get the expectation values for the individual optimizers
expectation_slsqp = expectation_values[:3]
expectation_bfgs = expectation_values[3:6]
expectation_nelder = expectation_values[6:9]
expectation_powell = expectation_values[9:12]

## Print the expectation values
print('The expectation values for SLSQP are:')
print(expectation_slsqp)
print()

print('The expectation values for P-BFGS are:')
print(expectation_bfgs)
print()

print('The expectation values for NELDER-MEAD are:')
print(expectation_nelder)
print()

print('The expectation values for POWELL are:')
print(expectation_powell)
print()

The expectation values for SLSQP are:
[array([[-8.278446e-07]]), array([[1.82685057e-06]]), array([[-8.278446e-07]])]

The expectation values for P-BFGS are:
[array([[-8.63430482e-09]]), array([[-1.67120046e-08]]), array([[-8.63430482e-09]])]

The expectation values for NELDER-MEAD are:
[array([[-1.56880831e-06]]), array([[-1.56880831e-06]]), array([[-1.56880831e-06]])]

The expectation values for POWELL are:
[array([[-3.07530503e-07]]), array([[-8.79740401e-09]]), array([[1.24151515e-06]])]



### For N = 3

### V = 1.1 (get the matrix and the zeromode)

In [16]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(3, 1.1)

The Schlogl operator matrix is:
[[  -0.275         2.95          0.            0.            0.
     0.            0.            0.        ]
 [   0.275        -3.225         5.9           0.            0.
     0.            0.            0.        ]
 [   0.            0.275       -11.62954545   11.82520661    0.
     0.            0.            0.        ]
 [   0.            0.            5.72954545  -28.46384298   23.70082645
     0.            0.            0.        ]
 [   0.            0.            0.           16.63863636  -56.70309917
    44.50206612    0.            0.        ]
 [   0.            0.            0.            0.           33.00227273
   -99.32252066   77.20413223    0.        ]
 [   0.            0.            0.            0.            0.
    54.82045455 -159.29731405  124.7822314 ]
 [   0.            0.            0.            0.            0.
     0.           82.09318182 -124.7822314 ]]

The volume parameter is:
1.1

The Hermitian form of the Schlogl matrix

### Estimate resources (fix at 5000 optimization iterations)

In [19]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter), NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
target_fidelity_threshold = 0.99

# Estimate min. ansatz depths
min_depths_N_3, zeromodes_N_3, resources_N_3 = estimate_resources(hermitian_matrix, zeromode, optimizers, num_qubits, target_fidelity_threshold)


Running VQD for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


SLSQP-RealAmplitudes - Depth 1: Average fidelity = 0.9999862960189679
Optimal depth 1 for SLSQP-RealAmplitudes achieved highest fidelity = [[0.99999262]]
Run number with highest fidelity: 3
Zeromode at optimal fidelity for SLSQP-RealAmplitudes: [[ 9.95772078e-01+0.j]
 [ 9.17990340e-02+0.j]
 [ 3.18618135e-03+0.j]
 [ 8.34191131e-04+0.j]
 [ 2.35897354e-04+0.j]
 [ 2.18597394e-05+0.j]
 [-3.24084472e-05+0.j]
 [-4.36898411e-05+0.j]]
Resource estimates for SLSQP-RealAmplitudes: Gate count = 9, Circuit depth = 5, Function calls = 241

Running VQD for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average fidelity = 0.999991221362102
Optimal depth 1 for SLSQP-TwoLocal achieved highest fidelity = [[0.99999143]]
Run number with highest fidelity: 6
Zeromode at optimal fidelity for SLSQP-TwoLocal: [[ 9.95790781e-01+0.j]
 [ 9.16013373e-02+0.j]
 [ 3.05225448e-03+0.j]
 [ 7.49298662e-04+0.j]
 [ 1.91679193e-04+0.j]
 [ 1.76291035e-05+0.j]
 [-6.18202840e-06+0.j]
 [-4.78518873e-07+0.j]]
Res

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 3: Average fidelity = 0.9991130539256284
Optimal depth 3 for SLSQP-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 9
Zeromode at optimal fidelity for SLSQP-EfficientSU2: [[-9.95667649e-01+0.j]
 [-9.28354087e-02+0.j]
 [-4.35197036e-03+0.j]
 [-2.09812567e-03+0.j]
 [-1.46770457e-03+0.j]
 [-1.08654751e-03+0.j]
 [-7.68217343e-04+0.j]
 [-5.03637812e-04+0.j]]
Resource estimates for SLSQP-EfficientSU2: Gate count = 21, Circuit depth = 13, Function calls = 1548

Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-EfficientSU2 - Depth 3: Average fidelity = 0.8991126720973626


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 4: Average fidelity = 0.7709007224674274


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 5: Average fidelity = 0.9999794517100227
Optimal depth 5 for P_BFGS-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 3
Zeromode at optimal fidelity for P_BFGS-EfficientSU2: [[-9.95669720e-01+0.j]
 [-9.28141326e-02+0.j]
 [-4.32733202e-03+0.j]
 [-2.09697440e-03+0.j]
 [-1.47236283e-03+0.j]
 [-1.09193427e-03+0.j]
 [-7.75346050e-04+0.j]
 [-5.10080121e-04+0.j]]
Resource estimates for P_BFGS-EfficientSU2: Gate count = 33, Circuit depth = 21, Function calls = 2489

Running VQD for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes
NELDER_MEAD-RealAmplitudes - Depth 1: Average fidelity = 0.7606095009978604
NELDER_MEAD-RealAmplitudes - Depth 2: Average fidelity = 0.7267996522992229
NELDER_MEAD-RealAmplitudes - Depth 3: Average fidelity = 0.5264141771740255
NELDER_MEAD-RealAmplitudes - Depth 4: Average fidelity = 0.5010778539486245
NELDER_MEAD-RealAmplitudes - Depth 5: Average fidelity = 0.3733735821250047
NELDER_MEAD-RealAmplitudes - De

### Print resource estimates

In [22]:
## Print the min. ansatz depths
print('The min. ansatz depths are:')
print(min_depths_N_3)
print()

## Print the resources required
print('The resources required are:')
print(resources_N_3)
print()

The min. ansatz depths are:
{'SLSQP-RealAmplitudes': 1, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 3, 'P_BFGS-RealAmplitudes': 2, 'P_BFGS-TwoLocal': 2, 'P_BFGS-EfficientSU2': 5, 'NELDER_MEAD-RealAmplitudes': 'Did not converge', 'NELDER_MEAD-TwoLocal': 'Did not converge', 'NELDER_MEAD-EfficientSU2': 'Did not converge', 'POWELL-RealAmplitudes': 'Did not converge', 'POWELL-TwoLocal': 'Did not converge', 'POWELL-EfficientSU2': 'Did not converge'}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 9, 'circuit_depth': 5, 'function_calls': 241}, 'SLSQP-TwoLocal': {'gate_count': 9, 'circuit_depth': 5, 'function_calls': 342}, 'SLSQP-EfficientSU2': {'gate_count': 21, 'circuit_depth': 13, 'function_calls': 1548}, 'P_BFGS-RealAmplitudes': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 5280}, 'P_BFGS-TwoLocal': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 5120}, 'P_BFGS-EfficientSU2': {'gate_count': 33, 'circuit_depth': 21, 'function_calls': 2489}, 'NELDER_ME

### Compute expectation values in the ground state

In [25]:
# Get the zeromodes
zeromodes = list(zeromodes_N_3.values())

# Initialize the list to store real-valued results
zeromodes_true = []

# Iterate through each entry in zeromodes
for arr in zeromodes:
    if isinstance(arr, str) and arr == "Did not converge":
        # Handle the case where the entry is the string 'Did not converge'
        zeromodes_true.append('Did not converge')
    else:
        # Convert the array to real numbers if it's not a string
        zeromodes_true.append([elem.real for elem in arr.flatten()])

# Initialize
expectation_values = []

# Compute 
for zeromode in zeromodes_true:
    if zeromode != "Did not converge":
        
        # Compute the expectation value in the ground state
        value = get_expectation(Q, np.abs(zeromode))
        expectation_values.append(value)
    else:
        expectation_values.append('No solution')

# Get the expectation values for the individual optimizers
expectation_slsqp = expectation_values[:3]
expectation_bfgs = expectation_values[3:6]
expectation_nelder = expectation_values[6:9]
expectation_powell = expectation_values[9:12]

## Print the expectation values
print('The expectation values for SLSQP are:')
print(expectation_slsqp)
print()

print('The expectation values for P-BFGS are:')
print(expectation_bfgs)
print()

print('The expectation values for NELDER-MEAD are:')
print(expectation_nelder)
print()

print('The expectation values for POWELL are:')
print(expectation_powell)
print()

The expectation values for SLSQP are:
[array([[-0.00333661]]), array([[-0.00393305]]), array([[6.28929599e-05]])]

The expectation values for P-BFGS are:
[array([[-0.00196205]]), array([[2.78240597e-05]]), array([[-6.09139538e-06]])]

The expectation values for NELDER-MEAD are:
['No solution', 'No solution', 'No solution']

The expectation values for POWELL are:
['No solution', 'No solution', 'No solution']



### For V = 5.5 (N = 3)

### Get the matrix and the zeromode

In [16]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(3, 5.5)

The Schlogl operator matrix is:
[[ -1.375        2.95         0.           0.           0.
    0.           0.           0.        ]
 [  1.375       -4.325        5.9          0.           0.
    0.           0.           0.        ]
 [  0.           1.375       -8.36590909   8.96900826   0.
    0.           0.           0.        ]
 [  0.           0.           2.46590909 -13.61673554  12.27603306
    0.           0.           0.        ]
 [  0.           0.           0.           4.64772727 -20.1964876
   15.94008264   0.           0.        ]
 [  0.           0.           0.           0.           7.92045455
  -28.22417355  20.08016529   0.        ]
 [  0.           0.           0.           0.           0.
   12.28409091 -37.81880165  24.81528926]
 [  0.           0.           0.           0.           0.
    0.          17.73863636 -24.81528926]]

The volume parameter is:
5.5

The Hermitian form of the Schlogl matrix is:
[[    3.78125      -10.003125       8.1125         0.
      

### Estimate resources (fix at 5000 optimization iterations)

In [20]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter), NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
target_fidelity_threshold = 0.99

# Estimate min. ansatz depths
min_depths_N_3, zeromodes_N_3, resources_N_3 = estimate_resources(hermitian_matrix, zeromode, optimizers, num_qubits, target_fidelity_threshold)


Running VQD for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


SLSQP-RealAmplitudes - Depth 1: Average fidelity = 0.9970638302177081
Optimal depth 1 for SLSQP-RealAmplitudes achieved highest fidelity = [[0.99999062]]
Run number with highest fidelity: 9
Zeromode at optimal fidelity for SLSQP-RealAmplitudes: [[-9.02193786e-01+0.j]
 [-4.19417810e-01+0.j]
 [-9.67704535e-02+0.j]
 [-2.57940563e-02+0.j]
 [-9.13272294e-03+0.j]
 [-4.22663860e-03+0.j]
 [-1.86845484e-03+0.j]
 [-6.75563609e-04+0.j]]
Resource estimates for SLSQP-RealAmplitudes: Gate count = 9, Circuit depth = 5, Function calls = 205

Running VQD for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average fidelity = 0.9999898995339059
Optimal depth 1 for SLSQP-TwoLocal achieved highest fidelity = [[0.99999468]]
Run number with highest fidelity: 8
Zeromode at optimal fidelity for SLSQP-TwoLocal: [[0.90202554+0.j]
 [0.41967907+0.j]
 [0.09709598+0.j]
 [0.02607235+0.j]
 [0.00935688+0.j]
 [0.00429987+0.j]
 [0.00218785+0.j]
 [0.00108488+0.j]]
Resource estimates for SLSQP-TwoLocal: Gat

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 3: Average fidelity = 0.9999961355580984
Optimal depth 3 for SLSQP-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 4
Zeromode at optimal fidelity for SLSQP-EfficientSU2: [[0.90162772+0.j]
 [0.42024658+0.j]
 [0.09793684+0.j]
 [0.02692136+0.j]
 [0.01018794+0.j]
 [0.00506009+0.j]
 [0.00309587+0.j]
 [0.00221386+0.j]]
Resource estimates for SLSQP-EfficientSU2: Gate count = 21, Circuit depth = 13, Function calls = 966

Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-TwoLocal - Depth 1: Average fidelity = 0.9999926103078364
Optimal depth 1 for P_BFGS-TwoLocal achieved highest fidelity = [[0.99999746]]
Run number with highest fidelity: 6
Zeromode at optimal fidelity for P_BFGS-TwoLocal: [[0.90186301+0.j]
 [0.41993165+0.j]
 [0.09741124+0.j]
 [0.02633061+0.j]
 [0.00953401+0.j]
 [0.00433282+0.j]
 [0.00242917+0.j]
 [0.00164237+0.j]]
Resource estimates for P_BFGS-TwoLocal: Gate count = 9, Circuit depth = 5, Function calls = 959

Running VQD for optimizer-ansatz pair: P_BFGS-EfficientSU2


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 1: Average fidelity = 0.9145635932628619


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average fidelity = 0.999990479540981
Optimal depth 2 for P_BFGS-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 2
Zeromode at optimal fidelity for P_BFGS-EfficientSU2: [[-0.90162581+0.j]
 [-0.42024938+0.j]
 [-0.09793962+0.j]
 [-0.02692733+0.j]
 [-0.01019482+0.j]
 [-0.00506577+0.j]
 [-0.00309902+0.j]
 [-0.00221526+0.j]]
Resource estimates for P_BFGS-EfficientSU2: Gate count = 15, Circuit depth = 9, Function calls = 1860

Running VQD for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes
NELDER_MEAD-RealAmplitudes - Depth 1: Average fidelity = 0.9999225775893927
Optimal depth 1 for NELDER_MEAD-RealAmplitudes achieved highest fidelity = [[0.99999067]]
Run number with highest fidelity: 7
Zeromode at optimal fidelity for NELDER_MEAD-RealAmplitudes: [[-9.02206500e-01+0.j]
 [-4.19390179e-01+0.j]
 [-9.67706008e-02+0.j]
 [-2.57933092e-02+0.j]
 [-9.13819314e-03+0.j]
 [-4.22892478e-03+0.j]
 [-1.89961861e-03+0.j]
 [-6.83084276e-04+0.j

### Print resource estimates

In [23]:
## Print the min. ansatz depths
print('The min. ansatz depths are:')
print(min_depths_N_3)
print()

## Print the resources required
print('The resources required are:')
print(resources_N_3)
print()

The min. ansatz depths are:
{'SLSQP-RealAmplitudes': 1, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 3, 'P_BFGS-RealAmplitudes': 2, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 2, 'NELDER_MEAD-RealAmplitudes': 1, 'NELDER_MEAD-TwoLocal': 1, 'NELDER_MEAD-EfficientSU2': 'Did not converge', 'POWELL-RealAmplitudes': 'Did not converge', 'POWELL-TwoLocal': 1, 'POWELL-EfficientSU2': 'Did not converge'}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 9, 'circuit_depth': 5, 'function_calls': 205}, 'SLSQP-TwoLocal': {'gate_count': 9, 'circuit_depth': 5, 'function_calls': 205}, 'SLSQP-EfficientSU2': {'gate_count': 21, 'circuit_depth': 13, 'function_calls': 966}, 'P_BFGS-RealAmplitudes': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 4430}, 'P_BFGS-TwoLocal': {'gate_count': 9, 'circuit_depth': 5, 'function_calls': 959}, 'P_BFGS-EfficientSU2': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 1860}, 'NELDER_MEAD-RealAmplitudes': {'gate_count': 9, 'circuit_depth': 

### Compute expectation values in the ground state

In [26]:
# Get the zeromodes
zeromodes = list(zeromodes_N_3.values())

# Initialize the list to store real-valued results
zeromodes_true = []

# Iterate through each entry in zeromodes
for arr in zeromodes:
    if isinstance(arr, str) and arr == "Did not converge":
        # Handle the case where the entry is the string 'Did not converge'
        zeromodes_true.append('Did not converge')
    else:
        # Convert the array to real numbers if it's not a string
        zeromodes_true.append([elem.real for elem in arr.flatten()])

# Initialize
expectation_values = []

# Compute 
for zeromode in zeromodes_true:
    if zeromode != "Did not converge":
        
        # Compute the expectation value in the ground state
        value = get_expectation(Q, np.abs(zeromode))
        expectation_values.append(value)
    else:
        expectation_values.append('No solution')

# Get the expectation values for the individual optimizers
expectation_slsqp = expectation_values[:3]
expectation_bfgs = expectation_values[3:6]
expectation_nelder = expectation_values[6:9]
expectation_powell = expectation_values[9:12]

## Print the expectation values
print('The expectation values for SLSQP are:')
print(expectation_slsqp)
print()

print('The expectation values for P-BFGS are:')
print(expectation_bfgs)
print()

print('The expectation values for NELDER-MEAD are:')
print(expectation_nelder)
print()

print('The expectation values for POWELL are:')
print(expectation_powell)
print()

The expectation values for SLSQP are:
[array([[-0.00414173]]), array([[-0.00299129]]), array([[-1.32308012e-05]])]

The expectation values for P-BFGS are:
[array([[2.57925068e-06]]), array([[-0.00189429]]), array([[4.93034982e-07]])]

The expectation values for NELDER-MEAD are:
[array([[-0.00417531]]), array([[-0.00417531]]), 'No solution']

The expectation values for POWELL are:
['No solution', array([[-0.00167871]]), 'No solution']



### For V = 10.5 (N = 3)

### Get the matrix and the zeromode

In [16]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(3, 10.5)

The Schlogl operator matrix is:
[[ -2.625        2.95         0.           0.           0.
    0.           0.           0.        ]
 [  2.625       -5.575        5.9          0.           0.
    0.           0.           0.        ]
 [  0.           2.625       -9.09642857   8.88265306   0.
    0.           0.           0.        ]
 [  0.           0.           3.19642857 -13.22193878  11.93061224
    0.           0.           0.        ]
 [  0.           0.           0.           4.33928571 -17.98418367
   15.07653061   0.           0.        ]
 [  0.           0.           0.           0.           6.05357143
  -23.41581633  18.35306122   0.        ]
 [  0.           0.           0.           0.           0.
    8.33928571 -29.5494898   21.79285714]
 [  0.           0.           0.           0.           0.
    0.          11.19642857 -21.79285714]]

The volume parameter is:
10.5

The Hermitian form of the Schlogl matrix is:
[[  13.78125     -22.378125     15.4875        0.         

### Estimate resources (fix at 5000 optimization iterations)

In [19]:
## Get the optimizers
maxiter = 5000
optimizers = [SLSQP(maxiter = maxiter), P_BFGS(maxfun = maxiter), NELDER_MEAD(maxiter = maxiter), POWELL(maxiter = maxiter)]

# Define parameters
dimension = Q.shape[0]
num_qubits = int(np.log2(dimension))
target_fidelity_threshold = 0.99

# Estimate min. ansatz depths
min_depths_N_3, zeromodes_N_3, resources_N_3 = estimate_resources(hermitian_matrix, zeromode, optimizers, num_qubits, target_fidelity_threshold)


Running VQD for optimizer-ansatz pair: SLSQP-RealAmplitudes


C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


SLSQP-RealAmplitudes - Depth 1: Average fidelity = 0.9332564492781412
SLSQP-RealAmplitudes - Depth 2: Average fidelity = 0.9995547418978692
Optimal depth 2 for SLSQP-RealAmplitudes achieved highest fidelity = [[0.99999997]]
Run number with highest fidelity: 3
Zeromode at optimal fidelity for SLSQP-RealAmplitudes: [[0.71212235+0.j]
 [0.63363979+0.j]
 [0.28190416+0.j]
 [0.10140715+0.j]
 [0.03683122+0.j]
 [0.01472893+0.j]
 [0.00664001+0.j]
 [0.00338365+0.j]]
Resource estimates for SLSQP-RealAmplitudes: Gate count = 15, Circuit depth = 9, Function calls = 807

Running VQD for optimizer-ansatz pair: SLSQP-TwoLocal
SLSQP-TwoLocal - Depth 1: Average fidelity = 0.9973013415103662
Optimal depth 1 for SLSQP-TwoLocal achieved highest fidelity = [[0.99869943]]
Run number with highest fidelity: 8
Zeromode at optimal fidelity for SLSQP-TwoLocal: [[0.7287945 +0.j]
 [0.62495586+0.j]
 [0.26306292+0.j]
 [0.08508621+0.j]
 [0.02940002+0.j]
 [0.02250055+0.j]
 [0.017423  +0.j]
 [0.01314473+0.j]]
Resource es

For Windows, using only current process. Multiple core use not supported.


SLSQP-EfficientSU2 - Depth 2: Average fidelity = 0.9999831308670764
Optimal depth 2 for SLSQP-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 10
Zeromode at optimal fidelity for SLSQP-EfficientSU2: [[-0.71210407+0.j]
 [-0.6336402 +0.j]
 [-0.28192027+0.j]
 [-0.10144598+0.j]
 [-0.03689226+0.j]
 [-0.01480824+0.j]
 [-0.00673068+0.j]
 [-0.00345505+0.j]]
Resource estimates for SLSQP-EfficientSU2: Gate count = 15, Circuit depth = 9, Function calls = 1005

Running VQD for optimizer-ansatz pair: P_BFGS-RealAmplitudes


...

P_BFGS-TwoLocal - Depth 1: Average fidelity = 0.9986970877834598
Optimal depth 1 for P_BFGS-TwoLocal achieved highest fidelity = [[0.99869717]]
Run number with highest fidelity: 4
Zeromode at optimal fidelity for P_BFGS-TwoLocal: [[0.72881975+0.j]
 [0.62494514+0.j]
 [0.26303143+0.j]
 [0.08506584+0.j]
 [0.02937521+0.j]
 [0.02248139+0.j]
 [0.01740417+0.j]
 [0.01312968+0.j]]
Resource estimates for P_BFGS-TwoLocal: Gate count = 9, Circuit depth = 5, Function calls = 1680

Running VQD for optimizer-ansatz pair: P_BFGS-EfficientSU2


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 1: Average fidelity = 0.8897812887223084


For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.
For Windows, using only current process. Multiple core use not supported.


P_BFGS-EfficientSU2 - Depth 2: Average fidelity = 0.9995629367191523
Optimal depth 2 for P_BFGS-EfficientSU2 achieved highest fidelity = [[1.]]
Run number with highest fidelity: 6
Zeromode at optimal fidelity for P_BFGS-EfficientSU2: [[0.71209804+0.j]
 [0.63364656+0.j]
 [0.28191904+0.j]
 [0.10144871+0.j]
 [0.03689795+0.j]
 [0.01481541+0.j]
 [0.00673189+0.j]
 [0.00345863+0.j]]
Resource estimates for P_BFGS-EfficientSU2: Gate count = 15, Circuit depth = 9, Function calls = 1110

Running VQD for optimizer-ansatz pair: NELDER_MEAD-RealAmplitudes
NELDER_MEAD-RealAmplitudes - Depth 1: Average fidelity = 0.9965594125083754
Optimal depth 1 for NELDER_MEAD-RealAmplitudes achieved highest fidelity = [[0.99893971]]
Run number with highest fidelity: 6
Zeromode at optimal fidelity for NELDER_MEAD-RealAmplitudes: [[0.72638172+0.j]
 [0.6254555 +0.j]
 [0.26641419+0.j]
 [0.08906198+0.j]
 [0.03258146+0.j]
 [0.02527247+0.j]
 [0.01923076+0.j]
 [0.01401398+0.j]]
Resource estimates for NELDER_MEAD-RealAmpli

### Print resource estimates

In [22]:
## Print the min. ansatz depths
print('The min. ansatz depths are:')
print(min_depths_N_3)
print()

## Print the resources required
print('The resources required are:')
print(resources_N_3)
print()

The min. ansatz depths are:
{'SLSQP-RealAmplitudes': 2, 'SLSQP-TwoLocal': 1, 'SLSQP-EfficientSU2': 2, 'P_BFGS-RealAmplitudes': 2, 'P_BFGS-TwoLocal': 1, 'P_BFGS-EfficientSU2': 2, 'NELDER_MEAD-RealAmplitudes': 1, 'NELDER_MEAD-TwoLocal': 1, 'NELDER_MEAD-EfficientSU2': 'Did not converge', 'POWELL-RealAmplitudes': 2, 'POWELL-TwoLocal': 1, 'POWELL-EfficientSU2': 'Did not converge'}

The resources required are:
{'SLSQP-RealAmplitudes': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 807}, 'SLSQP-TwoLocal': {'gate_count': 9, 'circuit_depth': 5, 'function_calls': 258}, 'SLSQP-EfficientSU2': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 1005}, 'P_BFGS-RealAmplitudes': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 1380}, 'P_BFGS-TwoLocal': {'gate_count': 9, 'circuit_depth': 5, 'function_calls': 1680}, 'P_BFGS-EfficientSU2': {'gate_count': 15, 'circuit_depth': 9, 'function_calls': 1110}, 'NELDER_MEAD-RealAmplitudes': {'gate_count': 9, 'circuit_depth': 5, 'function_ca

### Compute expectation values in the ground state

In [25]:
# Get the zeromodes
zeromodes = list(zeromodes_N_3.values())

# Initialize the list to store real-valued results
zeromodes_true = []

# Iterate through each entry in zeromodes
for arr in zeromodes:
    if isinstance(arr, str) and arr == "Did not converge":
        # Handle the case where the entry is the string 'Did not converge'
        zeromodes_true.append('Did not converge')
    else:
        # Convert the array to real numbers if it's not a string
        zeromodes_true.append([elem.real for elem in arr.flatten()])

# Initialize
expectation_values = []

# Compute 
for zeromode in zeromodes_true:
    if zeromode != "Did not converge":
        
        # Compute the expectation value in the ground state
        value = get_expectation(Q, np.abs(zeromode))
        expectation_values.append(value)
    else:
        expectation_values.append('No solution')

# Get the expectation values for the individual optimizers
expectation_slsqp = expectation_values[:3]
expectation_bfgs = expectation_values[3:6]
expectation_nelder = expectation_values[6:9]
expectation_powell = expectation_values[9:12]

## Print the expectation values
print('The expectation values for SLSQP are:')
print(expectation_slsqp)
print()

print('The expectation values for P-BFGS are:')
print(expectation_bfgs)
print()

print('The expectation values for NELDER-MEAD are:')
print(expectation_nelder)
print()

print('The expectation values for POWELL are:')
print(expectation_powell)
print()

The expectation values for SLSQP are:
[array([[-0.00015868]]), array([[-0.05321201]]), array([[-4.52762851e-06]])]

The expectation values for P-BFGS are:
[array([[-6.52333184e-06]]), array([[-0.05331227]]), array([[4.33193428e-09]])]

The expectation values for NELDER-MEAD are:
[array([[-0.03927206]]), array([[-0.03927206]]), 'No solution']

The expectation values for POWELL are:
[array([[0.01092014]]), array([[-0.05537051]]), 'No solution']

